# MS AI Hackathon Feb 2025

Project idea: Text to 3D demo

Stack:

LLM (Qwen 2.5 7B Instruct)

Hunyuan3D-2

Gradio Demo

https://huggingface.co/tencent/Hunyuan3D-2



## Install Hunyuan text to 3D

In [ ]:
# Install Hunyuan 3D 2
!git clone https://github.com/Tencent/Hunyuan3D-2.git
!pip install -r Hunyuan3D-2/requirements.txt
# for texture
%cd /content/Hunyuan3D-2/hy3dgen/texgen/custom_rasterizer
!python3 setup.py install
%cd /content/Hunyuan3D-2/hy3dgen/texgen/differentiable_renderer
!python3 setup.py install

Cloning into 'Hunyuan3D-2'...
remote: Enumerating objects: 584, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 584 (delta 176), reused 136 (delta 136), pack-reused 355 (from 1)
Receiving objects: 100% (584/584), 68.12 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (286/286), done.


## Initialize models

In [1]:
%cd /content/Hunyuan3D-2
from hy3dgen.rembg import BackgroundRemover
from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline, FaceReducer, FloaterRemover, DegenerateFaceRemover
from hy3dgen.text2image import HunyuanDiTPipeline

/content/Hunyuan3D-2


In [17]:
import torch
from PIL import Image

In [2]:
%cd /content/

/content


In [3]:
# Preload models
t2i = HunyuanDiTPipeline('Tencent-Hunyuan/HunyuanDiT-v1.1-Diffusers-Distilled')
i23d = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained('tencent/Hunyuan3D-2')
i23d_fast = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
        'tencent/Hunyuan3D-2',
        subfolder='hunyuan3d-dit-v2-0-fast',
        variant='fp16'
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.pag.pipeline_pag_hunyuandit.HunyuanDiTPAGPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2/hunyuan3d-dit-v2-0
Model path not exists, try to download from huggingface


Fetching 48 files:   0%|          | 0/48 [00:00<?, ?it/s]

/content/Hunyuan3D-2/hy3dgen/shapegen/pipelines.py:171: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2/hunyuan3d-dit-v2-0-fast
Model path not exists, try to download from huggingface


Fetching 48 files:   0%|          | 0/48 [00:00<?, ?it/s]

In [4]:
def text_to_image(prompt, filename='img.png'):
    global t2i
    image = t2i(prompt)
    image.save(filename)
    return image

def image_to_3d(image_path='assets/demo.png', image=None, filename='mesh.glb'):
    global i23d
    rembg = BackgroundRemover()
    #model_path = 'tencent/Hunyuan3D-2'
    if image is None:
        image = Image.open(image_path)

    if image.mode == 'RGB':
        image = rembg(image)

    #pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(model_path)

    mesh = i23d(image=image, num_inference_steps=30, mc_algo='mc',
                    generator=torch.manual_seed(2025))[0]
    mesh = FloaterRemover()(mesh)
    mesh = DegenerateFaceRemover()(mesh)
    mesh = FaceReducer()(mesh)
    mesh.export(filename)

    '''
    try:
        from hy3dgen.texgen import Hunyuan3DPaintPipeline
        pipeline = Hunyuan3DPaintPipeline.from_pretrained(model_path)
        mesh = pipeline(mesh, image=image)
        mesh.export('texture.glb')
    except Exception as e:
        print(e)
        print('Please try to install requirements by following README.md')
    '''
    return mesh

def text_to_3d(prompt='a car', filename='t2i_demo.glb'):
    global t2i, i23d
    rembg = BackgroundRemover()
    #t2i = HunyuanDiTPipeline('Tencent-Hunyuan/HunyuanDiT-v1.1-Diffusers-Distilled')
    #model_path = 'tencent/Hunyuan3D-2'
    #i23d = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(model_path)

    image = t2i(prompt)
    image = rembg(image)
    mesh = i23d(image, num_inference_steps=30, mc_algo='mc')[0]
    mesh = FloaterRemover()(mesh)
    mesh = DegenerateFaceRemover()(mesh)
    mesh = FaceReducer()(mesh)
    mesh.export(filename)


def image_to_3d_fast(image_path='assets/demo.png', image=None, filename='mesh.glb'):
    global i23d_fast
    rembg = BackgroundRemover()
    #model_path = 'tencent/Hunyuan3D-2'

    if image is None:
        image = Image.open(image_path)

    if image.mode == 'RGB':
        image = rembg(image)

    mesh = i23d_fast(image=image, num_inference_steps=30, mc_algo='mc',
                    generator=torch.manual_seed(2025))[0]
    mesh = FloaterRemover()(mesh)
    mesh = DegenerateFaceRemover()(mesh)
    mesh = FaceReducer()(mesh)
    mesh.export(filename)


In [5]:
# Load LLM (running locally, since I tried using OpenAI but ran into rate limit errors for single requests for some reason)
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

'\nprompt = "Give me a short introduction to large language model."\nmessages = [\n    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},\n    {"role": "user", "content": prompt}\n]\ntext = tokenizer.apply_chat_template(\n    messages,\n    tokenize=False,\n    add_generation_prompt=True\n)\nmodel_inputs = tokenizer([text], return_tensors="pt").to(model.device)\n\ngenerated_ids = model.generate(\n    **model_inputs,\n    max_new_tokens=512\n)\ngenerated_ids = [\n    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)\n]\n\nresponse = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]\n'

In [23]:
def generate_text(prompt, history):
    global model, tokenizer
    messages = []
    system_prompt = {"role": "system", "content": """Describe the object you want to generate in as much detail as possible. Consider the following aspects to create a precise and visually striking result:

    Object Type & Function: What is it? (e.g., an ancient sword, a futuristic helmet, a mystical artifact)
    Shape & Structure: What is its form? (e.g., sleek and aerodynamic, bulky and rugged, delicate and intricate)
    Materials & Texture: What is it made of? (e.g., polished steel, rough stone, translucent crystal, bio-organic matter)
    Color & Finish: What colors dominate the object, and how do they appear? (e.g., matte black with silver engravings, glowing emerald green, weathered bronze)
    Lighting & Reflection: How does it interact with light? (e.g., glossy and reflective, soft and diffused, neon-glowing edges)
    Decorative Elements: Are there engravings, patterns, symbols, or accessories? (e.g., intricate gold filigree, rune-like carvings, futuristic holographic displays)

    Respond with only the description, and do not ask any questions. The objective is to use the description to generate a 3D model.
    """}
    messages.append(system_prompt)
    for user_msg, bot_msg in history:
        if user_msg is not None:
            messages.append({"role": "user", "content": user_msg})
        if bot_msg is not None:
            if type(bot_msg) == tuple: # Ignore images in history
                continue
            messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": prompt})

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    print("Prompt:")
    print(text)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=150
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

## Demo

In [21]:
import gradio as gr
import json
import os
from datetime import datetime
from PIL import Image
import requests
from io import BytesIO

# Unused, testing
def generate_preview_image(prompt):
    """
    Generate a preview image based on the text prompt.
    In production, replace this with your actual text-to-image model.
    This is a placeholder that creates a simple colored rectangle.
    """
    # Placeholder image generation
    # In production, replace this with your text-to-image model
    img = Image.new('RGB', (512, 512), 'white')
    return img

# Unused, didn't have time
def handle_adjustment(adjustment, history):
    """
    Process adjustment requests and update the model
    """
    history.add_adjustment(adjustment)

    # Generate new preview based on adjustment
    updated_preview = generate_preview_image(adjustment)
    history.current_preview = updated_preview

    response = f"Applied adjustment: {adjustment}"
    history.append((None, response))

    return history, updated_preview

def handle_user_input(message, history):
    """
    Process user input and generate response/preview
    """
    response = generate_text(message, history)

    # Generate image
    filename = "image_"+str(len(history))+".png"
    image = text_to_image(response, filename=filename)

    history.append((message, None))
    history.append((None, (filename,)))
    history.append((None, response))

    return history, image

def submit_to_hunyuan(image):
    """
    Submit the final model to HunYuan for processing
    """
    #context = history.get_full_context()
    #image = history.current_preview
    filename = datetime.now().strftime("%Y%m%d%H%M%S")+".glb"
    pil_image = Image.fromarray(image) # Numpy image input
    image_to_3d_fast(image=pil_image, filename=filename)
    return filename

# Create the Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# 3D Model Generation Interface")
    gr.Markdown("Enter your model description, and I'll generate a preview. You can then make adjustments or submit to HunYuan.")

    with gr.Row():
        with gr.Column(scale=1):
            # Chat interface
            chatbot = gr.Chatbot(label="Model Generation Chat")
            msg = gr.Textbox(
                label="Enter your model description or adjustment request",
                placeholder="Describe the 3D model you want to create..."
            )

            with gr.Row():
                clear = gr.Button("Clear Chat")
                submit_btn = gr.Button("Generate 3D Model", variant="primary")

        with gr.Column(scale=1):
            # Preview area with larger size
            preview = gr.Image(label="Model Preview Image", height=512, width=512)
            result = gr.Model3D(label="Generated 3D Model")

    # Handle initial model generation
    msg.submit(
        fn=handle_user_input,
        inputs=[msg, chatbot],
        outputs=[chatbot, preview]
    )

    # Clear chat history
    clear.click(
        lambda: ([], None),
        outputs=[chatbot, preview]
    )

    # Handle final submission
    submit_btn.click(
        fn=submit_to_hunyuan,
        inputs=[preview],
        outputs=[result]
    )

# Launch the interface
if __name__ == "__main__":
    app.launch(debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cf735555f11d7befc6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/25 [00:00<?, ?it/s]

Using border_ratio from init: 0.15
Using guidance embed with scale 7.5


MC Level 0.0 Implicit Function:: 100%|██████████| 7134/7134 [00:17<00:00, 406.61it/s]


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://cf735555f11d7befc6.gradio.live
